In [1]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import ShotChartDetail # For player shot location and frequency
from nba_api.stats.endpoints import commonplayerinfo # For Common Player Information
import requests
import os
import numpy as np
#from nba_api.stats.endpoints import playergamelogs
import pandas as pd
def pull_rotation(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    frames = []
    for i in range(len(json["resultSets"])):
        data = json["resultSets"][i]["rowSet"]
        #print(data)
        columns = json["resultSets"][i]["headers"]
    #print(columns)

        df = pd.DataFrame.from_records(data, columns=columns)
        frames.append(df)
    gameframe = pd.concat(frames)
    return gameframe
def team_shotmap(team,season,stype ='Regular Season',league_id = '00',save_avg=False):

   
    #print(team_id)
    nba_teams = teams.get_teams()
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']
    print(team_list['ATL'])
        
    
    
    season_nullable= season
    team_id = team_list[team]
    print(team_id)
    shot_chart = ShotChartDetail(
                      player_id ='0',
                      
                      team_id=team_id,
                      season_nullable= season,
                      season_type_all_star=stype, 
                      context_measure_simple= 'FGA',
                                    league_id = league_id).get_data_frames()
        
    team_shots = shot_chart[0]
    if save_avg== True:
        
        avg = shot_chart[1]
        year = int(season.split('-')[0])+1
        avg.to_csv('team/'+str(year)+'/avg.csv',index=False)
    return team_shots
def pull_rotation(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    frames = []
    for i in range(len(json["resultSets"])):
        data = json["resultSets"][i]["rowSet"]
        #print(data)
        columns = json["resultSets"][i]["headers"]
    #print(columns)

        df = pd.DataFrame.from_records(data, columns=columns)
        frames.append(df)
    gameframe = pd.concat(frames)
    return gameframe
def get_shotrotations(season):
    nba_teams = teams.get_teams()
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']
    team_abr = full_name.keys()
    
    data = {}
    save_avg = True
    for team in team_abr:
        team = team.upper()
        shotmap = team_shotmap(team,season,save_avg=save_avg)
        save_avg=False
        games = list(set(shotmap.GAME_ID.tolist()))

        frames = []
        year =int(season.split('-')[0])+1
        path = 'rotations/'+str(year)
        team_id=team_list[team]
        
        if os.path.exists(path+'/'+str(team_id)+'.csv'):
            
            temp = pd.read_csv(path+'/'+str(team_id)+'.csv')
            stored = set(temp.GAME_ID.tolist())
            frames.append(temp)
            new_games = []
            for game in games:
                if str(game)[0:2] !='00':
                    new_game = '00'+str(game)
                    new_games.append(new_game)
                else:
                    new_games.append(game)
                    
            games = new_games
            
            new_stored = []
            for game in stored:
                if str(game)[0:2] !='00':
                    new_game = '00'+str(game)
                    new_stored.append(new_game)
                else:
                    new_stored.append(game)
            stored=new_stored
  
            games = list(set(games) -set(stored))
            #print(games)
        for game in games:
            url='https://stats.nba.com/stats/gamerotation?GameID='+game+'&LeagueID=00'
            df = pull_rotation(url)

            frames.append(df)
        all_games = pd.concat(frames)
        all_games = all_games.drop_duplicates()
        all_games['GAME_ID']=all_games['GAME_ID'].astype(str)
        all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])

        all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])


        shotmap['GAME_ID'] = np.where(shotmap['GAME_ID'].str[0:2]=='00', shotmap['GAME_ID'], '00'+shotmap['GAME_ID'])

        
        to_merge = all_games[['GAME_ID','TEAM_ID','PERSON_ID','IN_TIME_REAL','OUT_TIME_REAL']].reset_index(drop=True)
        to_merge['GAME_ID']=to_merge['GAME_ID'].astype(str)


        #to_merge.rename(columns={'PERSON_ID':'PLAYER_ID'},inplace=True)
        to_merge = to_merge[to_merge.TEAM_ID==team_id]        

        shotmap['SHOT_ID'] = shotmap['GAME_ID'].astype(str)+shotmap['GAME_EVENT_ID'].astype(str)
        shotmap['PERIOD'].value_counts()
        shotmap['time'] = (shotmap['PERIOD']-1)*720+(12-(shotmap['MINUTES_REMAINING']+1))*60+(60-shotmap['SECONDS_REMAINING'])
        shotmap['extra'] = (shotmap['PERIOD']//5) *(shotmap['PERIOD']-4)
        shotmap['time'] = shotmap['time']-(shotmap['extra']*420)
        #shotmap[shotmap.extra>0]
        shotmap['time']*=10
        shot_times = shotmap.merge(to_merge,on=['GAME_ID','TEAM_ID'],how='left')
        shot_times = shot_times[shot_times.time>=shot_times.IN_TIME_REAL]
        shot_times = shot_times[shot_times.time<shot_times.OUT_TIME_REAL]
        shot_times.sort_values(by='GAME_DATE')
        shot_times['SHOT_ID'] = shot_times['SHOT_ID'].astype(str)
        shot_times['PERSON_ID'] =shot_times['PERSON_ID'].astype(str)
        shot_on = shot_times.groupby('SHOT_ID')['PERSON_ID'].apply('|'.join).reset_index()
        shot_on.columns=['SHOT_ID','PLAYERS_ON']
        #print(shot_on)
        print(len(shot_on))
        value_dict={}
        id_count = []
        for col in shot_on.PLAYERS_ON.tolist():
            count = len(col.split('|'))
            if count not in value_dict:
                value_dict[count] = 1
            else:
                value_dict[count]+=1
            id_count.append(count)
        shot_on['id_count']=id_count
        
        print(value_dict)
        if 'PLAYERS_ON' in shotmap.columns:
            shotmap = shotmap.drop(columns='PLAYERS_ON')
        final_shotmap = shotmap.merge(shot_on,how='left').reset_index(drop=True)
        #print(final_shotmap['PLAYERS_ON'])
        final_shotmap['PLAYERS_ON'] = np.where(final_shotmap['id_count']==5, final_shotmap['PLAYERS_ON'],final_shotmap['PLAYER_ID'].astype(str))
        #print(final_shotmap['PLAYERS_ON'])
        final_shotmap.PLAYERS_ON = final_shotmap.PLAYERS_ON.astype(str)
        value_dict={}
        for col in final_shotmap.PLAYERS_ON.tolist():
            count = len(col.split('|'))
            if count not in value_dict:
                value_dict[count] = 1
            else:
                value_dict[count]+=1
        print(value_dict)
        print(len(shotmap))
        year =int(season.split('-')[0])+1
        final_shotmap = final_shotmap.drop(columns='id_count')
        final_shotmap.to_csv('team/'+str(year)+'/'+str(team_id)+'.csv',index = False)
        all_games = all_games[all_games.TEAM_ID==team_id]
        path = 'rotations/'+str(year)
        isExist = os.path.exists(path)
        if not isExist:

   # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
        all_games.to_csv(path+'/'+str(team_id)+'.csv',index=False)
        
        data[team]=final_shotmap
        #print(team)
        #print(len(shotmap))
        #print(len(final_shotmap))
        #print(len(shot_on))
    return data
data = get_shotrotations('2023-24')

1610612737
1610612737
5141
{5: 5141}
{5: 5141, 1: 3}
5144
1610612737
1610612738
4965
{5: 4965}
{5: 4965, 1: 2}
4967
1610612737
1610612739
4634
{5: 4606, 1: 19, 2: 9}
{5: 4606, 1: 92}
4698
1610612737
1610612740
4826
{5: 4826}
{5: 4826, 1: 3}
4829
1610612737
1610612741
4888
{5: 4825, 6: 28, 7: 35}
{5: 4825, 1: 79}
4904
1610612737
1610612742
4934
{5: 4839, 2: 37, 1: 37, 3: 21}
{5: 4839, 1: 98}
4937
1610612737
1610612743
4878
{5: 4878}
{5: 4878, 1: 5}
4883
1610612737
1610612744
4865
{5: 4865}
{5: 4865, 1: 6}
4871
1610612737
1610612745
4856
{5: 4772, 3: 13, 6: 24, 7: 8, 4: 8, 2: 8, 1: 23}
{5: 4772, 1: 89}
4861
1610612737
1610612746
4618
{5: 4618}
{5: 4618, 1: 4}
4622
1610612737
1610612747
4898
{5: 4835, 7: 9, 8: 7, 9: 20, 10: 20, 6: 7}
{5: 4835, 1: 65}
4900
1610612737
1610612748
4716
{5: 4716}
{5: 4716, 1: 3}
4719
1610612737
1610612749
4993
{5: 4993}
{5: 4993, 1: 5}
4998
1610612737
1610612750
4630
{5: 4478, 6: 19, 7: 16, 9: 25, 8: 4, 1: 9, 2: 51, 3: 28}
{5: 4478, 1: 161}
4639
1610612737
161

In [2]:
master= []
year = 2024
for team in teams.get_teams():
    team_id=team['id']
    df = pd.read_csv('team/'+str(year)+'/'+str(team_id)+'.csv')
    master.append(df)
all_shots = pd.concat(master)

In [3]:
for team in teams.get_teams():
    team_id=team['id']
    team_games = all_shots[all_shots.TEAM_ID==team['id']].GAME_ID.unique()
    team_games = list(team_games)
    opp_shots = all_shots[(all_shots.TEAM_ID!=team['id']) &(all_shots.GAME_ID.isin(team_games))].reset_index(drop=True)

    opp_shots['PLAYER_ID']='0'
    opp_shots['PLAYER_NAME']='Opponent'
    opp_shots['TEAM_ID']=team_id
    opp_shots.drop(columns=['PLAYERS_ON'],inplace=True)
    shotmap = opp_shots
                                        
    shotmap['GAME_ID'] = shotmap['GAME_ID'].astype(str)
    path = 'rotations/'+str(year)
    all_games = pd.read_csv(path+'/'+str(team_id)+'.csv')
    all_games = all_games.drop_duplicates()
    all_games = all_games[all_games.TEAM_ID==team_id]

    #print(all_games)
    
    all_games['GAME_ID']=all_games['GAME_ID'].astype(str)
    all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])

 
    shotmap['GAME_ID'] = np.where(shotmap['GAME_ID'].str[0:2]=='00', shotmap['GAME_ID'], '00'+shotmap['GAME_ID'])

    #print(len(shotmap))
    to_merge = all_games[['GAME_ID','TEAM_ID','PERSON_ID','IN_TIME_REAL','OUT_TIME_REAL']].reset_index(drop=True)
    to_merge['GAME_ID']=to_merge['GAME_ID'].astype(str)


    #to_merge.rename(columns={'PERSON_ID':'PLAYER_ID'},inplace=True)
    to_merge = to_merge[to_merge.TEAM_ID==team_id]        
    
    shot_times = shotmap.merge(to_merge,on=['GAME_ID','TEAM_ID'])
    #print(len(shot_times))
    shot_times = shot_times[shot_times.time>=shot_times.IN_TIME_REAL]
    shot_times = shot_times[shot_times.time<shot_times.OUT_TIME_REAL]
    #print(len(shot_times))
    shot_times.sort_values(by='GAME_DATE')
    shot_times['SHOT_ID'] = shot_times['SHOT_ID'].astype(str)
    shot_times['PERSON_ID'] =shot_times['PERSON_ID'].astype(str)
    shot_on = shot_times.groupby('SHOT_ID')['PERSON_ID'].apply('|'.join).reset_index()
    shot_on['SHOT_ID'] = shot_on['SHOT_ID'].astype(str)
    shotmap['SHOT_ID'] = shotmap['SHOT_ID'].astype(str)
    shot_on.columns=['SHOT_ID','PLAYERS_ON']
    #print(len(shot_on))
    value_dict = {}
    id_count=[]
    for col in shot_on.PLAYERS_ON.tolist():
        count = len(col.split('|'))
        if count not in value_dict:
            value_dict[count] = 1
        else:
            value_dict[count]+=1
        id_count.append(count)
    print(value_dict)
    shot_on['id_count'] = id_count
    final_shotmap = shotmap.merge(shot_on,how='left')
    final_shotmap['PLAYERS_ON'] = np.where(final_shotmap['id_count']==5, final_shotmap['PLAYERS_ON'],final_shotmap['PLAYER_ID'].astype(str))
    value_dict = {}
    id_count=[]
    for col in final_shotmap.PLAYERS_ON.tolist():
        count = len(col.split('|'))
        if count not in value_dict:
            value_dict[count] = 1
        else:
            value_dict[count]+=1
          
        id_count.append(count)
    print(value_dict)
    final_shotmap = final_shotmap.drop(columns='id_count')
    final_shotmap.to_csv('team/'+str(year)+'/'+str(team_id)+'vs.csv',index = False)
    #print(final_shotmap)
    #print(final_shotmap.head())


{5: 5030}
{5: 5030, 1: 7}
{5: 5083}
{5: 5083, 1: 6}
{5: 4554, 1: 17, 2: 11}
{5: 4554, 1: 91}
{5: 4825}
{5: 4825, 1: 2}
{5: 4689, 6: 29, 7: 25}
{5: 4689, 1: 68}
{5: 4893, 2: 42, 3: 16, 1: 36}
{5: 4893, 1: 100}
{5: 4786}
{5: 4786, 1: 2}
{5: 4805}
{5: 4805, 1: 4}
{5: 4661, 3: 10, 6: 28, 7: 7, 4: 6, 2: 5, 1: 14}
{5: 4661, 1: 72}
{5: 4718}
{5: 4718, 1: 3}
{5: 5140, 7: 16, 8: 7, 9: 17, 10: 21, 6: 8}
{5: 5140, 1: 76}
{5: 4726}
{5: 4726, 1: 4}
{5: 5202}
{5: 5202, 1: 7}
{5: 4638, 6: 11, 7: 18, 9: 25, 8: 4, 1: 9, 2: 47, 3: 32}
{5: 4638, 1: 155}
{5: 4810}
{5: 4810, 1: 7}
{5: 4696, 6: 26, 7: 8, 4: 13, 3: 8}
{5: 4696, 1: 60}
{5: 4675}
{5: 4675, 1: 5}
{5: 4956, 6: 52, 4: 10}
{5: 4956, 1: 70}
{5: 4666, 4: 28, 6: 23, 7: 1, 3: 5, 2: 10}
{5: 4666, 1: 72}
{5: 4946, 4: 12, 2: 17, 3: 10, 1: 1}
{5: 4946, 1: 45}
{5: 4710}
{5: 4710, 1: 3}
{5: 4653, 6: 20, 7: 51, 4: 12}
{5: 4653, 1: 88}
{5: 5027}
{5: 5027, 1: 4}
{5: 4749, 8: 13, 9: 38, 10: 9, 11: 12, 7: 11, 6: 2}
{5: 4749, 1: 89}
{5: 4894, 1: 69, 2: 1}
{5: 489

In [4]:
df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'SHOT_ID', 'time', 'extra', 'PLAYERS_ON'],
      dtype='object')

In [5]:
players = all_shots.PLAYER_ID.unique().tolist()
for player in players:
    df = all_shots[all_shots.PLAYER_ID==player]
    columns = ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM']
    df=df[columns]
    df.to_csv('2024/'+str(player)+'.csv',index=False)


In [6]:
def get_rotations(season,ps=False):
    nba_teams = teams.get_teams()
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']
    team_abr = full_name.keys()
    
    data = []
    save_avg = True
    stored=[]
    missed_games = []
    carry=''
    if ps==True:
        carry='ps'
    for team in team_abr:
        team = team.upper()
        team_id=team_list[team]
        year =int(season.split('-')[0])+1
        
        shotmap_url='https://raw.githubusercontent.com/gabriel1200/shot_data/master/team/'+str(year)+carry+'/'+str(team_id)+'.csv'
        
        response = requests.get(shotmap_url)
        if response.status_code != 404:

            shotmap = pd.read_csv(shotmap_url)
            save_avg=False
            games = list(set(shotmap.GAME_ID.tolist()))

            frames = []

            path = 'rotations/'+str(year)
            team_id=team_list[team]


            new_games = []
            for game in games:
                game = str(game)
                if game[0:2]!='00':
                    game = '00'+game
                    new_games.append(game)
                else:
                    new_games.append(games)
            games = new_games



            games = list(set(games) -set(stored))
            print(len(games))

            for game in games:
                game = str(game)
                if game[0:2]!='00':
                    game = '00'+game

                url='https://stats.nba.com/stats/gamerotation?GameID='+game+'&LeagueID=00'
                try:
                    df = pull_rotation(url)

                    frames.append(df)
                except ValueError:
                    print(game)
                    missed_games.append(game)
            
            if len(frames)!=0:
                all_games = pd.concat(frames)

                games =all_games['GAME_ID'].unique().tolist()
                stored =stored+games


                data.append(all_games)

        print(team)
            #print(len(shotmap))
            #print(len(final_shotmap))
            #print(len(shot_on))

    print(missed_games)
    return pd.concat(data)


In [7]:
'''
seasons = [str(year)+'-'+str(year+1)[-2:] for year in range (2004,2006)]

for season in seasons:
    year = int(season.split('-')[0])+1
    print(season)
    data = get_rotations(season)
   
    data.to_csv('rotations/'+str(year)+'.csv',index=False)
'''


"\nseasons = [str(year)+'-'+str(year+1)[-2:] for year in range (2004,2006)]\n\nfor season in seasons:\n    year = int(season.split('-')[0])+1\n    print(season)\n    data = get_rotations(season)\n   \n    data.to_csv('rotations/'+str(year)+'.csv',index=False)\n"

In [8]:
'''
seasons = [str(year)+'-'+str(year+1)[-2:] for year in range (2000,2023)]

for season in seasons:
    year = int(season.split('-')[0])+1
    print(season)
    data = get_rotations(season,ps=True)
   
    data.to_csv('rotations/'+str(year)+'ps.csv',index=False)
'''

"\nseasons = [str(year)+'-'+str(year+1)[-2:] for year in range (2000,2023)]\n\nfor season in seasons:\n    year = int(season.split('-')[0])+1\n    print(season)\n    data = get_rotations(season,ps=True)\n   \n    data.to_csv('rotations/'+str(year)+'ps.csv',index=False)\n"

In [9]:
'''
for year in range(2001,2006):
        path = 'rotations/'+str(year)
        
        isExist = os.path.exists(path)
        if not isExist:

   # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
        df =pd.read_csv(path+'.csv')
        teams = df.TEAM_ID.unique().tolist()
        
        for team in teams:
            teamdf=df[df.TEAM_ID==team]
            
            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)
        print(year)
'''

"\nfor year in range(2001,2006):\n        path = 'rotations/'+str(year)\n        \n        isExist = os.path.exists(path)\n        if not isExist:\n\n   # Create a new directory because it does not exist\n            os.makedirs(path)\n            print('Making Folder ' +path)\n        df =pd.read_csv(path+'.csv')\n        teams = df.TEAM_ID.unique().tolist()\n        \n        for team in teams:\n            teamdf=df[df.TEAM_ID==team]\n            \n            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)\n        print(year)\n"

In [10]:
'''
for year in range(2001,2006):
        path = 'rotations/'+str(year)+'ps'
        
        isExist = os.path.exists(path)
        if not isExist:

   # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
        df =pd.read_csv(path+'.csv')
        teams = df.TEAM_ID.unique().tolist()
        
        for team in teams:
            teamdf=df[df.TEAM_ID==team]
            
            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)
'''


"\nfor year in range(2001,2006):\n        path = 'rotations/'+str(year)+'ps'\n        \n        isExist = os.path.exists(path)\n        if not isExist:\n\n   # Create a new directory because it does not exist\n            os.makedirs(path)\n            print('Making Folder ' +path)\n        df =pd.read_csv(path+'.csv')\n        teams = df.TEAM_ID.unique().tolist()\n        \n        for team in teams:\n            teamdf=df[df.TEAM_ID==team]\n            \n            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)\n"